In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-finalexam-pp2/train.csv
/kaggle/input/2023-ml-finalexam-pp2/test.csv
/kaggle/input/2023-ml-finalexam-pp2/submit_sample.csv


In [2]:
#랜덤 시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
#데이터 불러오기
train = pd.read_csv("/kaggle/input/2023-ml-finalexam-pp2/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-finalexam-pp2/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-finalexam-pp2/submit_sample.csv")

In [4]:
#데이터 정보 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, 0 to label
dtypes: float64(784), object(1)
memory usage: 59.9+ MB


In [5]:
#y와 학습 데이터 셋 분리
y = train["label"]
train = train.drop(['label'],axis=1)

In [6]:
#y라벨 전처리
['5.0', '0.0', '4.0', '1.0', '9.0', '?', '3.0', '6.0', '7.0', '8.0']

y[y == '0.0'] = 0
y[y == '1.0'] = 1
y[y == '?'] = 2
y[y == '3.0'] = 3
y[y == '4.0'] = 4
y[y == '5.0'] = 5
y[y == '6.0'] = 6
y[y == '7.0'] = 7
y[y == '8.0'] = 8
y[y == '9.0'] = 9

In [7]:
#y를 숫자로!
y=y.astype("int64")

In [8]:
#pca를 위해 데이터 합치기
data = pd.concat([train, test],axis=0)

In [9]:
#pca진행
import numpy as np
from sklearn.decomposition import PCA


pca = PCA(n_components=20)
pca.fit(data)
train = pca.transform(train)
test = pca.transform(test)

In [10]:
#여러 모델 돌려보기!
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


lrc = LogisticRegression(max_iter = 2000)
lgc = LGBMClassifier()
rfc = RandomForestClassifier()
svc = SVC()
kmc = KNeighborsClassifier()
vc = VotingClassifier(estimators=[("lrc",lrc),('rfc',rfc),('svc',svc)])

models = [lrc, lgc, rfc, svc, kmc, vc]
names = ["lrc","lgc", "rfc", "svc", "kmc", 'vc']

for model, name in zip(models,names):
    model.fit(train, y)
    pred = model.predict(test)
    submit["Label"] = pred
    submit.to_csv("submit_%s.csv"%name,index=False)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c